#**Answer Generation 任務**
資料集引用台達閱讀理解資料集









### pytorch環境設定(https://pytorch.org)
選擇適合的版本並複製指令貼到本專案

In [1]:
!pip install torch torchvision

In [2]:
#確定有安裝成功
import torch
print(torch.__version__)
torch.cuda.is_available()

1.5.0+cu101


True

### **取得 google drive 存取權限**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration/

/content/drive/My Drive/Colab Notebooks/Bert/AnswerGeneration


In [5]:
!ls

albert	       test.json      trained_model2  train.json
predict.ipynb  trained_model  train.ipynb


In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

##下載Albert並放在albert目錄

In [7]:
!git clone https://github.com/harry83528/albert-zh-for-pytorch-transformers.git albert

fatal: destination path 'albert' already exists and is not an empty directory.


In [8]:
def use_model(model_name, config_file_path, model_file_path, vocab_file_path, num_labels):
    # 選擇模型並加載設定
    if(model_name == 'bert'):
        from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
        model_config, model_class, model_tokenizer = (BertConfig, BertForSequenceClassification, BertTokenizer)
        config = model_config.from_pretrained(config_file_path,num_labels = num_labels)
        model = model_class.from_pretrained(model_file_path, from_tf=bool('.ckpt' in 'bert-base-chinese'), config=config)
        tokenizer = model_tokenizer(vocab_file=vocab_file_path)
        return model, tokenizer
    elif(model_name == 'albert'):
        from albert.albert_zh import AlbertConfig, AlbertTokenizer, AlbertForMaskedLM
        model_config, model_class, model_tokenizer = (AlbertConfig, AlbertForMaskedLM, AlbertTokenizer)
        config = model_config.from_pretrained(config_file_path)
        #config = model_config.from_pretrained(config_file_path,num_labels = num_labels) # AlbertForMaskedLM 不需分類,不用設num_labels
        model = model_class.from_pretrained(model_file_path, config=config)
        tokenizer = model_tokenizer.from_pretrained(vocab_file_path)
        return model, tokenizer

### **指定想要用的模型是哪一種**
到以下網址查詢模型與函數資訊
https://huggingface.co/transformers

In [9]:
#通常我會在這設一些通用模型參數
# PRETRAINED_MODEL_Name ="bert-base-chinese"  # 指定 想要用的預訓練模型
trained_Model_Path = "trained_model" #選擇您儲存的model路徑

## **引入需要⽤到的函式庫**

In [10]:
import os
import pickle
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F # 激勵函數
import time

## **預測**

In [11]:
#predict 時把輸入的⽂字轉成輸入格式
def to_input_id(sentence_input):
    #return tokenizer.build_inputs_with_special_tokens(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))) # 此任務我們自己組special_token
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence_input))

In [18]:
#設定模型參數 Start =============================================================
#Step1:初始化加載模型 Start======================================================
model_setting = {
    "model_name":"albert", 
    "config_file_path": trained_Model_Path + "/config.json", 
    "model_file_path": trained_Model_Path + "/pytorch_model.bin", 
    "vocab_file_path":"albert/albert_tiny/vocab.txt",
    "num_labels": -1 # 分幾類(AlbertForMaskedLM 不需分類,不用設num_labels,在這隨便亂設)
}
#初始化加載模型
model, tokenizer = use_model(**model_setting)
#Step1:初始化加載模型 END========================================================

#Step2:指定硬體裝置 Start========================================================
# setting device
#你電腦的 GPU 能否被 PyTorch 調用,如果不行就使用CPU 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device",device)
model.to(device)
#Step2:指定硬體裝置 END==========================================================

model.eval()

#print("輸入您想輸入的context")
context ="10月之後，中國人民志願軍介入韓戰，這種政策受到挑戰。對外，倫敦擔憂與中國的敵對會危及英國在華利益，甚至促使共產陣營對香港等遠東殖民地構成威脅；對內，二戰後元氣大傷的英國因為武裝政策及軍費開銷而使國庫吃緊，衝擊民生及社會福利預算。而在韓國總統李承晚公開指責英國、加拿大、紐西蘭及澳大利亞等大英國協派兵國「影響麥克阿瑟遭到撤職」，之後，英國民眾對李氏及韓國第一共和政權的腐敗、專制產生極大反感，輿論立場也逐漸轉向反戰。工黨及艾德禮內閣的韓戰政策失去民心，黨內也因為輿論壓力及預算問題爆發分裂，這些因素使得工黨在1951年的國會大選中落敗。而選前抨擊工黨將英國拖入韓戰泥淖的保守黨，則在邱吉爾的帶領下取得執政權。邱吉爾上台後，迅速改變立場，推動停戰政策，使英國得以從朝鮮半島的難題抽身而出，同時在韓戰問題上不再支持美國。戰後的英國是聯合國軍組成諸國中最後一個於首都立碑紀念者，倫敦韓戰紀念碑在2013年11月奠基，2014年12月3日進行揭幕典禮。"
#context = input()
#print("輸入您想輸入的question")
#question = input()
question ="韓戰以後聯合國軍中哪一國是最後一個在首都立碑紀念的？"
input_id = to_input_id("[CLS]" + context + "[SEP]" + question + "[SEP][MASK]")

k = 20 # 取前k個有可能的答案

answer = ""
masked_index = len(input_id)-1 # 要預測的第一個字的索引

# 補齊長度512
while len(input_id)<512:
  input_id.append(0)

#假設答案長度最多15
count = 0
while(count < 15):
    print('input_id:',len(input_id),input_id)
    input_id_tensor=torch.tensor([input_id], dtype=torch.long)
    input_id_tensor = input_id_tensor.to(device) # 資料從CPU RAM 推到 GPU RAM
    outputs = model(input_id_tensor)
    #print('outputs',outputs)

    predictions = outputs[0]
    probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
    indicesList=indices.tolist()
    print('indicesList:',indicesList)
    predicted_tokens = tokenizer.convert_ids_to_tokens(indicesList)
    print('predicted_tokens',type(predicted_tokens),predicted_tokens)

    if predicted_tokens[0] == '[SEP]': #當生成出[SEP]結束
        break

    answer = answer + predicted_tokens[0]
    # 串接預測到的字並再最後的位置補上<MASK>的id 103
    input_id[masked_index] = indicesList[0]
    masked_index = masked_index + 1
    
    if masked_index < 512:
        input_id[masked_index] = 103
    else:
        break

    count += 1
    print('==='*20)

print("生成的答案:" + answer)

using device cuda
input_id: 512 [101, 8108, 3299, 722, 2527, 8024, 704, 1751, 782, 3696, 2562, 7544, 6725, 792, 1057, 7502, 2782, 8024, 6857, 4934, 3124, 5032, 1358, 1168, 2904, 2782, 511, 2205, 1912, 8024, 961, 3142, 3085, 2726, 5645, 704, 1751, 4638, 3147, 2205, 3298, 1314, 1350, 5739, 1751, 1762, 5836, 1164, 4660, 8024, 4493, 5635, 914, 886, 1066, 4496, 7369, 4245, 2205, 7676, 3949, 5023, 6895, 3346, 3658, 3696, 1765, 3539, 2768, 2014, 5545, 8039, 2205, 1058, 8024, 753, 2782, 2527, 1039, 3706, 1920, 1003, 4638, 5739, 1751, 1728, 4158, 3636, 6172, 3124, 5032, 1350, 6725, 6527, 7274, 7077, 5445, 886, 1751, 2430, 1391, 5215, 8024, 6128, 3080, 3696, 4495, 1350, 4852, 3298, 4886, 1164, 7521, 5050, 511, 5445, 1762, 7502, 1751, 5244, 5186, 3330, 2824, 3241, 1062, 7274, 2900, 6519, 5739, 1751, 510, 1217, 2897, 1920, 510, 5153, 6205, 5984, 1350, 4078, 1920, 1164, 765, 5023, 1920, 5739, 1751, 1295, 3836, 1070, 1751, 519, 2512, 7513, 7930, 1046, 7350, 4449, 6901, 1168, 3059, 5480, 520, 8024, 7

In [13]:
print('predict end')

predict end
